# Chapter 10

*Modeling and Simulation in Python*

Copyright 2021 Allen Downey

License: [Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International](https://creativecommons.org/licenses/by-nc-sa/4.0/)

In [1]:
# install Pint if necessary

try:
    import pint
except ImportError:
    !pip install pint

In [2]:
# download modsim.py if necessary

from os.path import exists

filename = 'modsim.py'
if not exists(filename):
    from urllib.request import urlretrieve
    url = 'https://raw.githubusercontent.com/AllenDowney/ModSim/main/'
    local, _ = urlretrieve(url+filename, filename)
    print('Downloaded ' + local)

In [3]:
# import functions from modsim

from modsim import *

# Case studies

This chapter presents case studies where you can practice the tools we have learned so far.

## Prehistoric World Population

On the Wikipedia page about world population estimates, the first table contains estimates for prehistoric populations.  The following cells process this table and plot some of the results.

In [2]:
import os

filename = 'World_population_estimates.html'

if not os.path.exists(filename):
    !wget https://raw.githubusercontent.com/AllenDowney/ModSimPy/master/data/World_population_estimates.html

In [3]:
from pandas import read_html

filename = 'World_population_estimates.html'
tables = read_html(filename, header=0, index_col=0, decimal='M')
len(tables)

Select `tables[1]`, which is the second table on the page.

In [4]:
table1 = tables[1]
table1.head()

Not all researchers provide estimates for the same dates.

Again, we'll replace the long column names with more convenient abbreviations.

In [6]:
table1.columns = ['PRB', 'UN', 'Maddison', 'HYDE', 'Tanton', 
                  'Biraben', 'McEvedy & Jones', 'Thomlinson', 'Durand', 'Clark']

Some of the estimates are in a form Pandas doesn't recognize as numbers, but we can coerce them to be numeric.

In [7]:
for col in table1.columns:
    table1[col] = pd.to_numeric(table1[col], errors='coerce')

Here are the results.  Notice that we are working in millions now, not billions.

In [8]:
table1.plot()
decorate(xlim=[-10000, 2000], xlabel='Year', 
         ylabel='World population (millions)',
         title='Prehistoric population estimates')
plt.legend(fontsize='small');

We can use `xlim` to zoom in on everything after Year 0.

In [15]:
table1.plot()
decorate(xlim=[0, 2000], xlabel='Year', 
         ylabel='World population (millions)',
         title='CE population estimates')

See if you can find a model that fits these data well from Year 0 to 1950.

How well does your best model predict actual population growth from 1950 to the present?

In [14]:
# Solution goes here

In [13]:
# Solution goes here

## One queue or two?

This case study is related to **queueing theory**, which is the study of systems that involve waiting in lines, also known as "queues".

Suppose you are designing the checkout area for a new store. There is
enough room in the store for two checkout counters and a waiting area
for customers. You can make two lines, one for each counter, or one line that feeds both counters.

In theory, you might expect a single line to be better, but it has some practical drawbacks: in order to maintain a single line, you might have to install barriers, and customers might be put off by what seems to be a longer line, even if it moves faster.

So you'd like to check whether the single line is really better and by
how much. Simulation can help answer this question.

This figure shows the three scenarios we'll consider:

![One queue, one server (left), one queue, two servers (middle), two
queues, two servers (right).](figs/queue.pdf){width="4.5in"}

As we did in the bike share model, we'll assume that a customer is equally likely to arrive during any time step. I'll denote this probability using the Greek letter lambda, $\lambda$, or the variable name `lam`. The value of $\lambda$ probably varies from day to day, so we'll have to consider a range of possibilities.

Based on data from other stores, you know that it takes 5 minutes for a customer to check out, on average. But checkout times are variable: most customers take less than 5 minutes, but some take substantially more. A simple way to model this variability is to assume that when a customer is checking out, they always have the same probability of finishing during the next time step, regardless of how long they have been checking out. I'll denote this probability using the Greek letter mu, $\mu$, or the variable name `mu`.

If we choose $\mu=1/5$ per minute, the average time for each checkout
will be 5 minutes, which is consistent with the data. Most people takes less than 5 minutes, but a few take substantially longer, which is probably not a bad model of the distribution in real stores.

Now we're ready to get started. In the repository for this book, you'll find a notebook called `queue.ipynb` that contains some code to get you started and instructions.

As always, you should practice incremental development: write no more
than one or two lines of code a time, and test as you go!

## Predicting salmon populations

Each year the U.S. Atlantic Salmon Assessment Committee reports
estimates of salmon populations in oceans and rivers in the northeastern United States. The reports are useful for monitoring changes in these populations, but they generally do not include predictions.

The goal of this case study is to model year-to-year changes in
population, evaluate how predictable these changes are, and estimate the probability that a particular population will increase or decrease in the next 10 years.

As an example, I use data from page 18 of the 2017 report, which
provides population estimates for the Narraguagus and Sheepscot Rivers
in Maine.

In the repository for this book, you'll find a notebook called
`salmon.ipynb` that contains some code to get you started and
instructions.

You should take my instructions as suggestions; if you want to try
something different, please do!

## Tree growth

This case study is based on "Height-Age Curves for Planted Stands of
Douglas Fir, with Adjustments for Density", a working paper by
Flewelling, Collier, Gonyea, Marshall, and Turnblom.

It provides "site index curves", which are curves that show the
expected height of the tallest tree in a stand of Douglas firs as a
function of age, for a stand where the trees are the same age.

Depending on the quality of the site, the trees might grow more quickly or slowing. So each curve is identified by a "site index\" that indicates the quality of the site.

The following figure shows site curves for three different site
indices. 

![Site index curves for tree
growth.](figs/trees-fig01.pdf){height="3in"}

The goal of this case study is to explain the shape of these
curves, that is, why trees grow the way they do.

As a starting place, let's assume that the ability of the tree to gain
mass is limited by the area it exposes to sunlight, and that the growth rate (in mass) is proportional to that area. In that case we can write:

$$m_{n+1} = m_n + \alpha A$$

where $m_n$ is the mass of the at time step $n$, $A$ is the area exposed to sunlight, and $\alpha$ is an unknown growth parameter.

To get from $m$ to $A$, I'll make the additional assumption that mass is proportional to height raised to an unknown power: $m = \beta h^D$ where $h$ is height, $\beta$ is an unknown constant of proportionality, and $D$ is the dimension that relates height and mass. I start by assuming $D=3$, but then revisit that assumption.

Finally, we'll assume that area is proportional to height squared:

$$A = \gamma h^2$$

I specify height in feet, and choose units for mass and area so that
$\beta=1$ and $\gamma=1$. Putting all that together, we can write a
difference equation for height:

$$h_{n+1}^D = h_n^D + \alpha h_n^2$$

With $D=3$, the solution to this equation is close to a straight line,
which is not a bad model for the growth curves. But the model implies
that trees can grow forever, and we know that's not true. As trees get
taller, it gets harder for them to move water and nutrients against the force of gravity, and their growth slows.

We can model this effect by adding a factor to the model similar to what we saw in the logistic model of population growth. Instead of assuming:

$$m_{n+1} = m_n + \alpha A$$

Let's assume

$$m_{n+1} = m_n + \alpha A (1 - h / K)$$

where $K$ is similar to the carrying capacity of the logistic model. As $h$ approaches $K$, the factor $(1 - h/K)$ goes to 0, causing growth to level off.

In the repository for this book, you'll find a notebook called
`trees.ipynb` that implements both models and uses them to fit the data.

There are no exercises in this case study; it is mostly meant as an
example of what you can do with the tools we have so far, and a preview of what we will be able to do with the tools in the next few chapters.